In [78]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

from pymongo import MongoClient
import pprint

In [2]:
deer_tier_url = 'http://deertier.com/Leaderboard/AnyPercentRealTime'

In [3]:
r = requests.get(deer_tier_url)

In [5]:
r.status_code

200

In [6]:
r.content

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head runat="server">\r\n    \r\n    <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png" />\r\n    <link rel="icon" type="image/png" href="/favicon-32x32.png" sizes="32x32" />\r\n    <link rel="icon" type="image/png" href="/favicon-16x16.png" sizes="16x16" />\r\n    <link rel="manifest" href="/manifest.json" />\r\n    <link rel="mask-icon" href="/safari-pinned-tab.svg" color="#fec200" />\r\n    <meta name="apple-mobile-web-app-title" content="Deer Tier" />\r\n    <meta name="application-name" content="Deer Tier" />\r\n    <meta name="theme-color" content="#331f52" />\r\n    <meta name="viewport" content="width=device-width" />\r\n\r\n    <link href="/CSS/Reset.css" rel="Stylesheet" type="text/css" />\r\n    <link href="/CSS/font-awesome.min.css" rel="Stylesheet" type="text/css"  />\r\n    <

In [9]:
client = MongoClient('mongodb://localhost:27017/')

In [10]:
db = client.metroid

In [11]:
pages = db.pages

In [12]:
pages.insert_one({'html': r.content})

In [13]:
soup = BeautifulSoup(r.content, "lxml")

In [14]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head runat="server">
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <link color="#fec200" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="Deer Tier" name="apple-mobile-web-app-title"/>
  <meta content="Deer Tier" name="application-name"/>
  <meta content="#331f52" name="theme-color"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="/CSS/Reset.css" rel="Stylesheet" type="text/css"/>
  <link href="/CSS/font-awesome.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/CSS/Styles.css?v=21" rel="Stylesheet" type="text/css"/>
  <script src="/Scripts/jq

In [76]:
div = soup.find("div", {"class": "scoreTable"})
table = div.find("table")

# This returns an iterator over the rows in the table.
rows = table.find_all("tr")

all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "rank": None, "player": None, "time": None, "comment": None
}

# The first row contains header information, so we are skipping it.
for row in rows[1:]:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    columns = row.find_all("td")
    new_row['rank'] = int(columns[0].text.strip())
#     new_row['rank'] = int(columns[0].text.strip())
#     new_row['player'] = columns[1].text.strip()
#     new_row['time'] = columns[2].text.strip()
#     new_row['comment'] = columns[4].text.strip()
#     all_rows.append(new_row)

IndexError: list index out of range

In [77]:
import json
import re
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

In [79]:
API_KEY = 'a670f41f909ce02de585b95a55fb8e9a'

In [80]:
def breweries_near_me(citystate):
    """Function to query the beermapping.com API for brewerys need a given city, state
    
        Input(str): A string of the city and state in the format phoenix,az
        
        Output(dict): returns a dictionary with a 
            key 'brewery_results' and v
            value: a list of dictionaries of brewery specific info
    
    """
    #Create default dict for results from the API
    brewery_ratings = []
    
    #Format input for API consumption
    form_city_state = citystate.lower().replace(' ','')
    
    #Make a call to the beer mapping API for the user specified city, st.
    breweries_content = requests.get('http://beermapping.com/webservice/loccity/{}/{}'.format(API_KEY, 
                                                                                              form_city_state))

    #Make soup specific object from request API
    content = BeautifulSoup(breweries_content.content, 'html.parser')
    
    #Iterate through all results from the API request
    for brew_name, brew_id, status, zip_, phone in zip(content.findAll('name'), \
                                                       content.findAll('id'), \
                                                       content.findAll('status'), \
                                                       content.findAll('zip'), \
                                                       content.findAll('phone')):

        #Skip beer store results to only return brewery information
        if status.string != 'Beer Store':
            temp_dict = dict()
            temp_dict['name'] = brew_name.string
            temp_dict['phone'] = phone.string
            temp_dict['zip'] = zip_.string
            temp_dict['rating'] = brew_rating(brew_id.string)
            brewery_ratings.append(temp_dict)
        else:
            pass
        
    return brewery_ratings

In [81]:
def brew_rating(brewery_id):
    """Helper function to breweries_near_me
    
    
        Input(str): Brewery ID
        
        Output(dict): various ratings associated with the user defined brewery
    """
    rating_dict = {}
    
    brew_rating = requests.get('http://beermapping.com/webservice/locscore/{}/{}'.format(API_KEY,
                                                                                         brewery_id))
    content = BeautifulSoup(brew_rating.content, 'html.parser')
    rating_dict['overall'] = content.find('overall').string
    rating_dict['selection'] = content.find('selection').string
    rating_dict['service'] = content.find('service').string
    rating_dict['atmosphere'] = content.find('atmosphere').string
    rating_dict['food'] = content.find('food').string
    rating_dict['reviewcount'] = content.find('reviewcount').string
    
    return rating_dict

In [82]:
brewery_list = breweries_near_me('phoenix,az')

In [86]:
brewery_list[1]

{'name': u"BJ's Brewhouse - Desert Ridge",
 'phone': u'(480) 538-0555',
 'rating': {'atmosphere': u'0',
  'food': u'0',
  'overall': u'0',
  'reviewcount': u'0',
  'selection': u'0',
  'service': u'0'},
 'zip': u'85050'}

In [83]:
# import MongoDB modules
from pymongo import MongoClient
#from bson.objectid import ObjectId

# connect to the hosted MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client.beer_db

In [84]:
collection = db.breweries

In [87]:
for brewery in brewery_list:

    if not collection.find_one(brewery['name']):
        collection.insert_one(brewery)

In [91]:
for pub in collection.find():
    print(pub)

{u'phone': u'(480) 563-9273', u'_id': ObjectId('59bd7d3a66a1611c690bf313'), u'name': u'Yard House - Phoenix', u'zip': u'85050', u'rating': {u'atmosphere': u'0', u'selection': u'0', u'service': u'0', u'food': u'0', u'overall': u'0', u'reviewcount': u'0'}}
{u'phone': u'(480) 538-0555', u'_id': ObjectId('59bd7d3a66a1611c690bf314'), u'name': u"BJ's Brewhouse - Desert Ridge", u'zip': u'85050', u'rating': {u'atmosphere': u'0', u'selection': u'0', u'service': u'0', u'food': u'0', u'overall': u'0', u'reviewcount': u'0'}}
{u'phone': u'(480) 753-4453', u'_id': ObjectId('59bd7d3a66a1611c690bf315'), u'name': u'Arriba Mexican Grill - Ahwatukee', u'zip': u'85048', u'rating': {u'atmosphere': u'0', u'selection': u'0', u'service': u'0', u'food': u'0', u'overall': u'0', u'reviewcount': u'0'}}
{u'phone': u'(602) 522-2288', u'_id': ObjectId('59bd7d3a66a1611c690bf316'), u'name': u'Delux', u'zip': u'85016', u'rating': {u'atmosphere': u'0', u'selection': u'0', u'service': u'0', u'food': u'0', u'overall': u'0

In [102]:
url = 'https://etherscan.io/txs'

In [103]:
page = requests.get(url)

In [104]:
page.status_code

200

In [105]:
page.text

u'<!doctype html>\n<html lang="en">\n<head><title>\nEthereum Transactions Information\n</title>\n<meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="Description" content="The Ethereum BlockChain Explorer, API and Analytics Platform" /><meta name="author" content="etherscan.io" /><meta name="keywords" content="ethereum, explorer, ether, search, blockchain, crypto, currency" /><meta name="format-detection" content="telephone=no" />\n<script src="/cdn-cgi/apps/head/gWQ5_2Bt4EXQyZe_2Ddss068hJ4.js"></script><script type="text/javascript" src="/assets/plugins/jquery/jquery.min.js"></script>\n<script type="text/javascript" src="/jss/jquery-ui.min.js"></script>\n<style type="text/css">\r\n    div.tooltip-inner {\r\n        max-width: 430px;\r\n        background-color: #505050;\r\n        font-size: 11px;\r\n    }\r\n</style>\n<link rel="shortcut icon" href="/images/favicon2.ico" /><link rel="stylesheet" type="text/css" href="//fonts.goog